In [1]:
using JuMP
using Gurobi
using DataFrames

### Option 1
It uses disjunctive constraints but it takes about 5 minutes with Gurobi.

In [2]:
LOSS = DataFrame(lot = Int.(collect(range(1, 11, length = 11))), kg = [43, 26, 37, 28, 13, 54, 62, 49, 19, 28, 30])
LOSS_DIC = Dict(eachrow(LOSS))

LIFE_SPAN = DataFrame(lot = Int.(collect(range(1, 11, length = 11))), life = [8, 8, 2, 8, 4, 8, 8, 8, 6, 8, 8])
LIFE_SPAN_DIC = Dict(eachrow(LIFE_SPAN))

N_LOT = 11
LINES = 3
PROCESSING_TIME = 2;
M = N_LOT * PROCESSING_TIME;

In [3]:
model = Model(Gurobi.Optimizer)

# start time
@variable(model, s[1:N_LOT] >= 0)

# 1 if LOT i is processed by LINE j
@variable(model, isProcess[1:N_LOT, 1:LINES], Bin)

# 1 if LOT i after LOT j
@variable(model, after[1:N_LOT, 1:N_LOT], Bin)
for j in 1:N_LOT
    for k in 1:N_LOT
        if k < j
            delete(model, after[j, k])
        end
    end
end


# a LOT can only be processed by one LINE
for i in 1:N_LOT
    @constraint(model, sum(isProcess[i, j] for j in 1:LINES) == 1)
end

# no overlapping processes of the LOTs per LINE
for i in 1:LINES
    for j in 1:N_LOT
        for k in 1:N_LOT
            if k > j
                # LOT j is processed after LOT k is finished processing
                @constraint( model, s[j] >= s[k] + PROCESSING_TIME - M * (1 - after[j, k]) - M * (1 - isProcess[j, i]) - M * (1 - isProcess[k, i]) )
                
                # or LOT k is processed after LOT j is finished processing
                @constraint( model, s[k] >= s[j] + PROCESSING_TIME - M * after[j, k] - M * (1 - isProcess[j, i]) - M * (1 - isProcess[k, i]) )
            end
        end
    end
end

# finish time before life span
for j in 1:N_LOT
    @constraint(model, s[j] + PROCESSING_TIME <= LIFE_SPAN_DIC[j])
end

@objective(model, Min, sum((s[j] + PROCESSING_TIME) * LOSS_DIC[j] for j in 1:N_LOT));

In [4]:
optimize!(model)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 352 rows, 110 columns and 1694 nonzeros
Model fingerprint: 0x2329cc5b
Variable types: 11 continuous, 99 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 2026.0000000
Presolve removed 11 rows and 12 columns
Presolve time: 0.01s
Presolved: 341 rows, 98 columns, 1623 nonzeros
Variable types: 10 continuous, 88 integer (88 binary)

Root relaxation: objective 7.780000e+02, 68 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  778.00000    0   15 2026.00000  778.00000  61.6%     -    0s
H    0     0                    1820.0000000  778.00000  57.3%     -    0s
H    0     0                    1808.0000000  778.00000 

In [5]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [6]:
value.(s)'

1×11 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 2.0  6.0  0.0  6.0  2.0  0.0  0.0  2.0  4.0  4.0  4.0

In [7]:
replace(value.(isProcess)', -0.0 => -, 0.0 => -, 0.0 => -)

3×11 Array{Any,2}:
 1.0   -    -    -    -    -   1.0   -    -   1.0   -
  -   1.0   -    -    -   1.0   -   1.0  1.0   -    -
  -    -   1.0  1.0  1.0   -    -    -    -    -   1.0

In [8]:
getobjectivevalue(model)

1620.0

### Option 2
It uses the sequence-position.

In [9]:
model2 = Model(Gurobi.Optimizer)

# Maximum number of SLOTs per LINE
N_SLOT = Int(ceil(N_LOT / LINES))

# If LOT i assigned to time SLOT j
@variable(model2, x[1:N_LOT, 1:N_SLOT], Bin)

# One LOT per time SLOT
for i in 1:N_LOT
    @constraint(model2, sum(x[i,j] for j in 1:N_SLOT) == 1)
end

# At most the number of lines per time SLOT
for j in 1:N_SLOT
    @constraint(model2, sum(x[i,j] for i in 1:N_LOT) <= LINES)
end

# Process LOT before life span
for i in 1:N_LOT
    @constraint(model2, sum(x[i, j] * j for j in 1:N_SLOT) <= LIFE_SPAN_DIC[i] / PROCESSING_TIME)
end

# Objective
@objective(model2, Min, sum(x[i,j] * j * PROCESSING_TIME * LOSS_DIC[i] for i in 1:N_LOT for j in 1:N_SLOT));

In [10]:
optimize!(model2)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 26 rows, 44 columns and 132 nonzeros
Model fingerprint: 0x5e560143
Variable types: 0 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 1998.0000000
Presolve removed 13 rows and 8 columns
Presolve time: 0.00s
Presolved: 13 rows, 36 columns, 72 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)

Root relaxation: objective 1.620000e+03, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1620.0000000 1620.00000  0.00%     -    0s

Explored 0 nodes (17 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 1620 1998 

Opti

In [11]:
termination_status(model2)

OPTIMAL::TerminationStatusCode = 1

In [12]:
replace(value.(x), -0.0 => -, 0.0 => -)

11×4 Array{Any,2}:
  -   1.0   -    -
  -    -    -   1.0
 1.0   -    -    -
  -    -   1.0   -
  -   1.0   -    -
 1.0   -    -    -
 1.0   -    -    -
  -   1.0   -    -
  -    -   1.0   -
  -    -    -   1.0
  -    -   1.0   -

In [13]:
termination_status(model2)

OPTIMAL::TerminationStatusCode = 1

In [14]:
li = 1
total_loss = 0
for s in 1:N_SLOT
    for w in 1:N_LOT
        if value.(x[w, s]) == 1.0
            println("Wagon $(w) in time slot $(s) in line $(li): total loss $(PROCESSING_TIME * s * LOSS_DIC[w])")
            total_loss += value.(x[w, s]) * s * PROCESSING_TIME * LOSS_DIC[w]
            li += 1
        end
    end
    li = 1
end
println("--------------------------")
println("Total Loss: ", total_loss)

Wagon 3 in time slot 1 in line 1: total loss 74
Wagon 6 in time slot 1 in line 2: total loss 108
Wagon 7 in time slot 1 in line 3: total loss 124
Wagon 1 in time slot 2 in line 1: total loss 172
Wagon 5 in time slot 2 in line 2: total loss 52
Wagon 8 in time slot 2 in line 3: total loss 196
Wagon 4 in time slot 3 in line 1: total loss 168
Wagon 9 in time slot 3 in line 2: total loss 114
Wagon 11 in time slot 3 in line 3: total loss 180
Wagon 2 in time slot 4 in line 1: total loss 208
Wagon 10 in time slot 4 in line 2: total loss 224
--------------------------
Total Loss: 1620.0
